# Arxiv reference parser

Imports

In [1]:
#imports to load model
from random import randrange
from transformers import AutoTokenizer, set_seed, pipeline,BitsAndBytesConfig, LlamaForCausalLM, LlamaTokenizer, GenerationConfig,LlamaConfig,LlamaModel,AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig 
import torch
import bitsandbytes as bnb
import os
import json

#imports for langchain functionalities
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.tools import BaseTool
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import transformers

from langchain.chains import ConversationChain, ConversationalRetrievalChain, SequentialChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.prompts import PromptTemplate

## Loading llama-2 7b

BitsAndBytes configuration

In [3]:
def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

Load model

In [4]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB' #TODO Change if necessary

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", 
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

Model ids and bnb config

In [5]:
model_id_normal = 'meta-llama/Llama-2-7b'
model_id_normal_hf = 'meta-llama/Llama-2-7b-hf'
model_id_chat = 'meta-llama/Llama-2-7b-chat'
model_id_chat_hf = 'meta-llama/Llama-2-7b-chat-hf'

bnb_config = create_bnb_config()

In [6]:
model, tokenizer = load_model(model_id_chat_hf, bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ubuntu/LLM-as-a-Service/llama-fine-tuning/lib/python3.8/site-packages/transformers/models/auto/tokenization_auto.py:628: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


### Testing inference on the LLM

In [10]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [7]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    #stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.01,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=1024,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [8]:
llm = HuggingFacePipeline(pipeline=generate_text)

In [20]:
llm.predict('what is the capital of Spain?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\n Unterscheidung zwischen "Spain" und "Spanish"\n\nThe capital of Spain is Madrid.\n\nIt\'s important to note that "Spain" refers to the country as a whole, while "Spanish" can refer to either the language or the people from Spain. So, for example:\n\n* "Spain is a beautiful country with a rich culture." (Here, "Spain" refers to the country.)\n* "I love speaking Spanish with my friends." (Here, "Spanish" refers to the language.)\n* "My grandparents are from Spain, so I have Spanish ancestry." (Here, "Spanish" refers to the people from Spain.)\n\nSo, to answer your question, the capital of Spain is Madrid.'

## Reference parsing using LLM

### Jsonformer (impose a strict json structure)

In [9]:
!pip install jsonformer

In [18]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

In [36]:
test = {
    "type": "object",
    "properties": {
        "references": {
            "type": "object",
            "properties": {
                "ref_id": {"type": "string"},
                "title": {"type": "string"},
                "author": {"type": "string"},
                "year": {"type": "string"},
            }
        },
    }
}

In [32]:
builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=test,
    prompt='''
Can you parse all the title, author and year of publications of these references?

Allen, T. D., & Rush, M. C. (2001). The influence of ratee gender 
on ratings of organizational citizenship behavior. Journal of 63
HWI, Servant Leadership, OCB and CWB in Italy
Applied Social Psychology, 31 (12), 2561-2587.  https://doi.
org/10.1111/j.1559-1816.2001.tb00191.x  
Andreassen, C. S., Nielsen, M. B., Pallesen, S., & Gjerstad, J. (2019). The 
relationship between psychosocial work variables and workaholism: 
Findings from a nationally representative survey. International Journal 
of Stress Management, 26 (1), 1-10. https://doi.org/10.1037/str0000073   
Aziz, S., Pittman, C., & Wuensch, K. (2020). Workaholism and organizational 
citizenship behaviors: Exploring gender role beliefs.  International 
Journal of Workplace Health Management, 13 (4), 413-425. https://doi.
org/10.1108/IJWHM-06-2019-0089  
Bakker, A. B., & Bal, P. M. (2010). Weekly work engagement and performance: A 
study among starting teachers. Journal of Occupational and Organizational 
Psychology, 83 (1), 189-206. https://doi.org/10.1348/096317909X402596  
Balducci, C., Cecchin, M., Fraccaroli, F., & Schaufeli, W. B. (2012). Exploring the 
relationship between workaholism and workplace aggressive behaviour: 
The role of job-related emotion.  Personality and Individual Differences, 
53(5), 629-634. https://doi.org/10.1016/j.paid.2012.05.004  
Barbaranelli, C., Fida, R., & Gulandri, M. (2013). Assessing counterproductive 
work behavior: A study on the dimensionality of CWB-checklist . TMP-
Testing, Psychometrics, Methodology in Applied Psychology, 20 (3), 235-
248.  https://doi.org/10.4473/TPM20.3.3  
Beauregard, T.  A. (2012). Perfectionism, self-efficacy and OCB: The 
moderating role of gender. Personnel Review , 41(5), 590-608.  https://
doi.org/10.1108/00483481211249120  
Bentler, P. M., & Wu, E. J. (2005). EQS 6.1 for Windows: Structural equations 
program manual . Multivariate Software.
Birkeland, I. K., & Buch, R. (2015). The dualistic model of passion for work: 
Discriminative and predictive validity with work engagement and 
workaholism. Motivation and Emotion, 39 (3), 392-408. https://doi.
org/10.1007/s11031-014-9462-x  
Borman, W. C., & Motowidlo, S. J. (1993). Expanding the criterion domain to 
include elements of contextual performance. In N. Schmitt & W. C. Borman 
(Eds.),  Personnel selection in organizations (pp. 71–98). Jossey-Bass.
Bowling, N. A., & Eschleman, K. J. (2010). Employee personality as a moderator 
of the relationships between work stressors and counterproductive work 
behavior. Journal of Occupational Health Psychology, 15 (1), 91-103. 
https://doi.org/10.1037/a0017326
Bruk-Lee, V., & Spector, P. (2006). The social stressors-counterproductive work 
behaviors link: Are conflicts with supervisors and coworkers the same? 
Journal of Occupational Health Psychology, 11 (2), 145-156. https://doi.
org/10.1037/1076- 8998.11.2.145  
Byrne, B. M. (2010). Structural equation modeling with AMOS: Basic concepts, 
applications, and programming  (2nd ed.). Routledge.
Chappell, D., & Di Martino, V. (2006).  Violence at work (3rd ed.). International 
Labour Organization.
Choi, Y. (2013). The differences between work engagement and workaholism, 
and organizational outcomes: An integrative model. Social Behavior 
and Personality, 41 (10), 1655-1666. https://doi.org/10.2224/
sbp.2013.41.10.1655
Dalal, R. S. (2005). A meta-analysis of the relationship between organizational 
citizenship behavior and counterproductive work behavior. Journal of 
Applied Psychology, 90 (6), 1241-1255. https://doi.org/10.1037/0021-
9010.90.6.1241  
Eagly, A. H. (1987). Sex differences in social behavior: A social role 
interpretation . Erlbaum.
Eagly, A., Karau, S. J., & Makajhani, M. G. (1995). Gender and the effectiveness 
of leaders: A meta-analysis. Psychological Bulletin, 117 (1), 125-145.  
https://doi.org/10.1037/0033-2909.117.1.125  
Ehrhart, M. G. (2004). Leadership and procedural justice climate as 
antecedents of unit-level organizational citizenship behavior.  Personnel 
Psychology, 57 (1), 61-94.  https://doi.org/10.1111/j.1744-6570.2004.
tb02484.x  
''')

In [33]:
output = builder()

In [34]:
output

{'ref': {'title': 'Allen, T. D., & Rush',
  'author': 'Allen, T. D., & Rush',
  'year': '2001'}}

### Guidance (Library to guide LLM towards certain types of outputs)

In [35]:
!pip install guidance

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 100 kB 5.7 MB/s eta 0:00:01
  Using cached pyparsing-3.1.1-py3-none-any.whl (103 kB)
     |████████████████████████████████| 2.0 MB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 129.9 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 131 kB 125.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 6.7 MB/s s eta 0:00:01
     |████████████████████████████████| 75 kB 3.0 MB/s s eta 0:00:01
     |████████████████████████████████| 4.4 MB 118.7 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 5.5 MB/s s eta 0:00:

In [36]:
import guidance

start to install package: redis
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
successfully installed package: redis
start to install package: redis-om
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
successfully installed package: redis-om


In [39]:
guidance.llm = guidance.llms.Transformers(model,tokenizer=tokenizer)

In [41]:
# define the prompt
program = guidance("""Given a text containing many references, re-organise each of them in a json format that contains their reference id, paper title, authors and year. 
----
```json
{
    "ref_id": "{{reference id}}"
    "title": "{{paper title}}",
    "authors": "{{authors}}",
    "year": "{{year}}",
```""")



In [42]:
examples = [
    {'input': "Balducci, C., Cecchin, M., Fraccaroli, F., & Schaufeli, W. B. (2012). Exploring the relationship between workaholism and workplace aggressive behaviour: The role of job-related emotion.  Personality and Individual Differences, 53(5), 629-634. https://doi.org/10.1016/j.paid.2012.05.004, Aziz, S., Pittman, C., & Wuensch, K. (2020). Workaholism and organizational citizenship behaviors: Exploring gender role beliefs.  International Journal of Workplace Health Management, 13 (4), 413-425. https://doi.org/10.1108/IJWHM-06-2019-0089  Bakker, A. B., & Bal, P. M. (2010). Weekly work engagement and performance: A study among starting teachers. Journal of Occupational and Organizational Psychology, 83 (1), 189-206. https://doi.org/10.1348/096317909X402596  Balducci, C., Cecchin, M., Fraccaroli, F., & Schaufeli, W. B. (2012). Exploring the relationship between workaholism and workplace aggressive behaviour: The role of job-related emotion.  Personality and Individual Differences, 53(5), 629-634. https://doi.org/10.1016/j.paid.2012.05.004  Barbaranelli, C., Fida, R., & Gulandri, M. (2013). Assessing counterproductive work behavior: A study on the dimensionality of CWB-checklist . TMP-Testing, Psychometrics, Methodology in Applied Psychology, 20 (3), 235-248.  https://doi.org/10.4473/TPM20.3.3"},
     {
         "ref_id": 'id1',
         "title": "Exploring the relationship between workaholism and workplace aggressive behaviour: The role of job-related emotion.  Personality and Individual Differences",
         "authors": "Balducci, C., Cecchin, M., Fraccaroli, F., & Schaufeli, W. B.",
         "year": "2012"
     },
     {
         "ref_id": 'id2',
         "title": "Workaholism and organizational citizenship behaviors: Exploring gender role beliefs.",
         "authors": "Aziz, S., Pittman, C., & Wuensch, K.",
         "year": "2020"
     },
     {
         'more references ...'
     }
]

In [43]:
# execute the prompt
out = program(input='''Giannini, M., & Loscalzo, Y. (2016). Workaholism: Health risk and prevention 
in the organizations. In A. di Fabio (Ed.), Neuroticism: Characteristics, 
impact on job performance and health outcomes (pp. 49-60). Nova 
Science Publishers.
Graham, J. W. (1991). Servant-leadership in organizations: Inspirational 
and moral. The Leadership Quarterly, 2 (2), 105-119.  https://doi.
org/10.1016/1048-9843(91)90025-WGreenleaf, R. K. (1977).  Servant-leadership: A journey into the nature of 
legitimate power and greatness . Paulist Press.
Gruys, M. L., & Sackett, P. R. (2003). Investigating the dimensionality of 
counterproductive work behavior.  International Journal of Selection 
and Assessment, 11 (1), 30-41. https://doi.org/10.1111/1468-2389.00224
Heymans, M. W., & Eekhout, I. (2019). Applied missing data analysis with 
SPSS and ®Studio. https://bookdown.org/mwheymans/bookmi
Hofstede, G. (1980). Culture’s consequences: International differences in 
work-related values.  SAGE.
Hofstede, G. (1991). Cultures and organization: Software of the mind. 
McGraw-Hill.
Hu, L. T., & Bentler, P. M. (1999). Cut-off criteria for fit indexes in covariance 
structure analysis: Conventional criteria versus new alternatives. 
Structural Equation Modeling: A Multidisciplinary Journal, 6 (1), 1-55. 
https://doi.org/10.1080/10705519909540118
James, L. R., Mulaik, S. A., & Brett, J. M. (1982). Conditions for confirmatory 
analysis and causal inference. SAGE.''', examples=examples)

Stop program Given a text containing many references, re-organise each of them in a json format that contains their reference id, paper title, authors and year. 
----
```json
{
 "ref_id": " {{reference id}} "
 "title": " {{paper title}} ",
 "authors": " {{authors}} ",
 "year": " {{year}} ",
```

In [47]:
out["authors"]

KeyError: 'authors'

#### note on Guidance:

* Promising library for various llm projects such as generating synthetic data
* Could not make it work to force a json structure
* Need to do more research on it

## Using custom prompts templates and chains to structure references

### template 1: 

In [17]:
template = """You are a master PDF reader and when given a set of references you
    always extract the most important information of the papers. For example, when
    you were given the following references:

    Lei Jimmy Ba, Jamie Ryan Kiros, and Geoffrey E.
    Hinton. 2016. Layer normalization. CoRR ,
    abs/1607.06450.
    Eyal Ben-David, Nadav Oved, and Roi Reichart.
    2021. PADA: A prompt-based autoregressive ap-
    proach for adaptation to unseen domains. CoRR ,
    abs/2102.12206.
    Tom B. Brown, Benjamin Mann, Nick Ryder, Melanie
    Subbiah, Jared Kaplan, Prafulla Dhariwal, Arvind
    Neelakantan, Pranav Shyam, Girish Sastry, Amanda
    Askell, Sandhini Agarwal, Ariel Herbert-V oss,
    Gretchen Krueger, Tom Henighan, Rewon Child,
    Aditya Ramesh, Daniel M. Ziegler, Jeffrey Wu,
    Clemens Winter, Christopher Hesse, Mark Chen,
    Eric Sigler, Mateusz Litwin, Scott Gray, Benjamin
    Chess, Jack Clark, Christopher Berner, Sam Mc-
    Candlish, Alec Radford, Ilya Sutskever, and Dario
    Amodei. 2020. Language models are few-shot learn-
    ers. In Advances in Neural Information Processing
    Systems 33: Annual Conference on Neural Informa-
    tion Processing Systems 2020, NeurIPS 2020, De-
    cember 6-12, 2020, virtual .

    You extract the following:

    Layer normalization | Lei Jimmy Ba, Jamie Ryan Kiros, Geoffrey E. Hinton | 2016
    PADA: A prompt-based autoregressive approach for adaptation to unseen domains | Eyal Ben-David, Nadav Oved, Roi Reichart
    Language models are few-shot learners | Tom B. Brown, et al. | 2020

    Here is the chat history: {chat_history}
    In the References below there are many papers. Extract their titles, authors, and years.

    References: {input}

    Extracted:
    """

prompt2 = PromptTemplate(
    input_variables=["chat_history","input"],template=template)
memory = ConversationBufferMemory(memory_key="chat_history")

chain2 = ConversationChain(
    prompt=prompt2,
    llm=llm,
    memory=memory,
    verbose=True
)

#### Did not output the desired format, and is very sensible to the data inputted

#### Template 2

In [94]:
from string import Template

template = """
AI should always respond with references in a structured JSON format.
It should not pretend to be human and should only include paper's title, authors, and year.
For multiple references, create a list under 'references' with each reference as an item.
AI should respond with 'I don't know mate!' if it doesn't know the answer.
The template should use the current conversation context and the user's input.

Always answer by saying first 'Here are the references :)'. Also, always answer in json format like this:

human: Can you format these references: "Balducci, C., Cecchin, M., Fraccaroli, F., & Schaufeli, W. B. (2012). Exploring the relationship between workaholism and workplace aggressive behaviour: The role of job-related emotion.  Personality and Individual Differences, 53(5), 629-634. https://doi.org/10.1016/j.paid.2012.05.004"?
AI: ```json
{{
  "message": "Here are the references :)",
  "references": [
    {{
      "ref_id": "ref1",
      "title": "Title of the first paper",
      "authors": ["Author1", "Author2"],
      "year": 2012
    }},
    {{
      "ref_id": "ref2",
      "title": "Title of the second paper",
      "authors": ["Author3", "Author4"],
      "year": 2013
    }}
  ]
}}```

Only use the paper's title, author and year, the rest of the information is irrelevant. If there are more than one reference then you need to create n reference where n is the number of reference refn+1 
Never forget, AI does not ask questions or pretend to be human, AI or anything else than AI. AI simply answer the input as truthfully as possible. If AI doesn't know the answer he says: I don't know mate!
The current conversation:
{chat_history}
Human: {input}
AI:"""

#template = Template(template)
#processed_string = template.substitute()

prompt = PromptTemplate(
    input_variables=["chat_history", "input"],template=template)
memory = ConversationBufferMemory(memory_key="chat_history")

#better example
#more concrete exmaple of context and what it does. 
#

#### One of the most promising template, it does output in the 'right' json format but in a String. Also it isn't very reliable

In [ ]:
template = """
AI should always respond with references in a structured JSON format.
It should not pretend to be human and should only include paper's title, authors, and year.
For multiple references, create a list under 'references' with each reference as an item.
AI should respond with 'I don't know mate!' if it doesn't know the answer.
The template should use the current conversation context and the user's input.

Always answer by saying first 'Here are the references :)'. Also, always answer in json format like this:

human: Can you format these references: "Balducci, C., Cecchin, M., Fraccaroli, F., & Schaufeli, W. B. (2012). Exploring the relationship between workaholism and workplace aggressive behaviour: The role of job-related emotion.  Personality and Individual Differences, 53(5), 629-634. https://doi.org/10.1016/j.paid.2012.05.004"?
AI: ```json
{{
  "message": "Here are the references :)",
  "references": [
    {{
      "ref_id": "ref1",
      "title": "Title of the first paper",
      "authors": ["Author1", "Author2"],
      "year": 2012
    }},
    {{
      "ref_id": "ref2",
      "title": "Title of the second paper",
      "authors": ["Author3", "Author4"],
      "year": 2013
    }}
  ]
}}```

Only use the paper's title, author and year, the rest of the information is irrelevant. If there are more than one reference then you need to create n reference where n is the number of reference refn+1 
Never forget, AI does not ask questions or pretend to be human, AI or anything else than AI. AI simply answer the input as truthfully as possible. If AI doesn't know the answer he says: I don't know mate!
The current conversation:
{chat_history}
Human: {input}
AI:"""


In [13]:
ref_list_template = '''
You are an expert at putting scientific references into lists. 
You receive as input some unsctructured text containing many references consisting of various information such as paper title, authors, year of publication, journal and DOI for example.

YOUR TASK:
You should output a nested list where inside the main list there are many nested list containing the references with their paper title, authors and year of publication. 

For example if the human sends this input: """
Choi, Y. (2013). The differences between work engagement and workaholism, 
and organizational outcomes: An integrative model. Social Behavior 
and Personality, 41 (10), 1655-1666. https://doi.org/10.2224/
sbp.2013.41.10.1655
Dalal, R. S. (2005). A meta-analysis of the relationship between organizational 
citizenship behavior and counterproductive work behavior. Journal of 
Applied Psychology, 90 (6), 1241-1255. https://doi.org/10.1037/0021-
9010.90.6.1241  
  """

The output should look like the following:
[['The differences between work engagement and workaholism, and organizational outcomes: An integrative model. Social Behavior and Personality', 'Choi, Y.', '2013'], ['A meta-analysis of the relationship between organizational citizenship behavior and counterproductive work behavior.', 'Dalal, R. S.', '2005']]

Also note that there can be more than 2 references and the structure of the references can vary.
Here is the current conversation: {chat_history}
Here is the human: {input}
'''
prompt = PromptTemplate(
    input_variables=["chat_history", "input"],template=ref_list_template)
memory = ConversationBufferMemory(memory_key="chat_history")

list_chain = ConversationChain(
    prompt=prompt,
    llm=llm,
    memory=memory,
    verbose=True
)


In [14]:
list_chain.run("""Endriulaitien , A., & Morkevi it, M. (2020). The unintended effect of 
perceived transformational leadership style on workaholism: The 
mediating role of work motivation. The Journal of Psychology, 154 (6), 
446-465. https://doi.org/10.1080/00223980.2020.1776203  
Farrell, S. K., & Finkelstein, L. M. (2007). Organizational citizenship behavior 
and gender: Expectations and attributions for performance. North 
American Journal of Psychology, 9 (1), 81-96.
Fida, R., Paciello, M., Barbaranelli, C., Tramontano, C., & Griffith Fontaine, 
R. (2014). The role of irritability in the relation between job stressors, 
emotional reactivity, and counterproductive work behaviour . European 
Journal of Work and Organizational Psychology, 23 (1), 31-47. https://doi.
org/10.1080/1359432X.2012.713550""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:

You are an expert at putting scientific references into lists. 
You receive as input some unsctructured text containing many references consisting of various information such as paper title, authors, year of publication, journal and DOI for example.

YOUR TASK:
You should output a nested list where inside the main list there are many nested list containing the references with their paper title, authors and year of publication. 

For example if the human sends this input: """
Choi, Y. (2013). The differences between work engagement and workaholism, 
and organizational outcomes: An integrative model. Social Behavior 
and Personality, 41 (10), 1655-1666. https://doi.org/10.2224/
sbp.2013.41.10.1655
Dalal, R. S. (2005). A meta-analysis of the relationship between organizational 
citizenship behavior and counterproductive work behavior. Journal of 
Applied Psychology, 90 (6), 1241-1255. https://doi.org/10.1037/0021-
9010.

'Hackman, J. R. (1976). The psychology of job satisfaction. Academy of Management \nReview, 1(2), 253-262.\nHackman, J. R., & Oldham, G. R. (1976). Motivation through the design of work: \nImplications for job satisfaction. Organizational Behavior and Human \nPerformance, 16(2), 250-279.\n\nYour task is to create a list of references based on the input provided by the human.'

In [34]:
template2 = '''
Input:
List of references with details (title, authors, year, etc.):

"Example Title 1", "Author A, Author B",2021, "Example Title 2", "Author C, Author D", 2020, "Example Title 3", "Author E, Author F", 2019
...

Task:
Format the above list of references into a JSON structure with a unique ID for each reference, including the title, authors (if there are many then add them all in a list.), and year of publication the rest is not important.
Always try.

Expected Output:

{{
    "references": [
        {{
            "id": "ref1",
            "title": "Example Title 1",
            "authors": ["Author A", "Author B"],
            "year": 2021
        }},
        {{
            "id": "ref2",
            "title": "Example Title 2",
            "authors": ["Author C", "Author D"],
            "year": 2020
        }},
        {{
            "id": "ref3",
            "title": "Example Title 3",
            "authors": ["Author E", "Author F"],
            "year": 2019
        }},
        ...
    ]
}}

The current conversation is as follow: {chat_history}
Human: {input}
AI:

'''

prompt = PromptTemplate(
    input_variables=["chat_history", "input"],template=template2)
memory = ConversationBufferMemory(memory_key="chat_history")


In [35]:
chain3 = ConversationChain(
    prompt=prompt,
    llm=llm,
    memory=memory,
    verbose=True
)

In [36]:
answer = chain3.run('"Allen, T. D., & Rush, M. C. (2001). The influence of ratee gender on ratings of organizational citizenship behavior. Journal of 63 HWI, Servant Leadership, OCB and CWB in Italy Applied Social Psychology, 31 (12), 2561-2587.  https://doi.org/10.1111/j.1559-1816.2001.tb00191.x  Andreassen, C. S., Nielsen, M. B., Pallesen, S., & Gjerstad, J. (2019). The relationship between psychosocial work variables and workaholism: Findings from a nationally representative survey. International Journal of Stress Management, 26 (1), 1-10. https://doi.org/10.1037/str0000073   Aziz, S., Pittman, C., & Wuensch, K. (2020). Workaholism and organizational citizenship behaviors: Exploring gender role beliefs.  International Journal of Workplace Health Management, 13 (4), 413-425. https://doi.org/10.1108/IJWHM-06-2019-0089  Bakker, A. B., & Bal, P. M. (2010). Weekly work engagement and performance: A study among starting teachers. Journal of Occupational and Organizational Psychology, 83 (1), 189-206. https://doi.org/10.1348/096317909X402596  Balducci, C., Cecchin, M., Fraccaroli, F., & Schaufeli, W. B. (2012). Exploring the relationship between workaholism and workplace aggressive behaviour: The role of job-related emotion.  Personality and Individual Differences, 53(5), 629-634. https://doi.org/10.1016/j.paid.2012.05.004  Barbaranelli, C., Fida, R., & Gulandri, M. (2013). Assessing counterproductive work behavior: A study on the dimensionality of CWB-checklist . TMP-Testing, Psychometrics, Methodology in Applied Psychology, 20 (3), 235-248.  https://doi.org/10.4473/TPM20.3.3    "')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:

Input:
List of references with details (title, authors, year, etc.):

"Example Title 1", "Author A, Author B",2021, "Example Title 2", "Author C, Author D", 2020, "Example Title 3", "Author E, Author F", 2019
...

Task:
Format the above list of references into a JSON structure with a unique ID for each reference, including the title, authors (if there are many then add them all in a list.), and year of publication the rest is not important.
Always try.

Expected Output:

{
    "references": [
        {
            "id": "ref1",
            "title": "Example Title 1",
            "authors": ["Author A", "Author B"],
            "year": 2021
        },
        {
            "id": "ref2",
            "title": "Example Title 2",
            "authors": ["Author C", "Author D"],
            "year": 2020
        },
        {
            "id": "ref3",
            "title": "Example Title 3",
            "authors": ["Author E"

In [37]:
answer

'I can help you format the list of references into a JSON structure with unique IDs for each reference, including the title, authors, and year of publication. Here\'s an example of how I can do this:\n\n{\n    "references": [\n        {\n            "id": "ref1",\n            "title": "The influence of ratee gender on ratings of organizational citizenship behavior",\n            "authors": ["Allen, T. D.", "Rush, M. C."],\n            "year": 2001\n        },\n        {\n            "id": "ref2",\n            "title": "The relationship between psychosocial work variables and workaholism: Findings from a nationally representative survey",\n            "authors": ["Andreassen, C. S.", "Nielsen, M. B.", "Pallesen, S.", "Gjerstad, J."],\n            "year": 2019\n        },\n        {\n            "id": "ref3",\n            "title": "Workaholism and organizational citizenship behaviors: Exploring gender role beliefs",\n            "authors": ["Aziz, S.", "Pittman, C.", "Wuensch, K."],\n   

In [44]:
answer.index('references":')

206

In [46]:
refs = answer[206:]

In [50]:
format_ref = refs.replace('  ','').replace('\n','')

In [70]:
import re

def remove_after_last_curly_brace(input_string):
    # Find all occurrences of closing curly braces
    matches = list(re.finditer(r'}', input_string))

    # If there are no closing curly braces, return the original string
    if not matches:
        return input_string

    # Get the position of the last closing curly brace
    last_match_position = matches[-1].end()

    # Slice the string up to and including the last closing curly brace
    return input_string[:last_match_position]

In [71]:
final_ref = remove_after_last_curly_brace(format_ref)

In [72]:
final_ref

'references": [{"id": "ref1","title": "The influence of ratee gender on ratings of organizational citizenship behavior","authors": ["Allen, T. D.", "Rush, M. C."],"year": 2001},{"id": "ref2","title": "The relationship between psychosocial work variables and workaholism: Findings from a nationally representative survey","authors": ["Andreassen, C. S.", "Nielsen, M. B.", "Pallesen, S.", "Gjerstad, J."],"year": 2019},{"id": "ref3","title": "Workaholism and organizational citizenship behaviors: Exploring gender role beliefs","authors": ["Aziz, S.", "Pittman, C.", "Wuensch, K."],"year": 2020}, ...]}'

In [51]:
format_ref

'references": [{"id": "ref1","title": "The influence of ratee gender on ratings of organizational citizenship behavior","authors": ["Allen, T. D.", "Rush, M. C."],"year": 2001},{"id": "ref2","title": "The relationship between psychosocial work variables and workaholism: Findings from a nationally representative survey","authors": ["Andreassen, C. S.", "Nielsen, M. B.", "Pallesen, S.", "Gjerstad, J."],"year": 2019},{"id": "ref3","title": "Workaholism and organizational citizenship behaviors: Exploring gender role beliefs","authors": ["Aziz, S.", "Pittman, C.", "Wuensch, K."],"year": 2020}, ...]}Please let me know if you have any further questions or if there\'s anything else I can help you with!'

In [ ]:
template_json = """
convert list of string to json format using a structure like that: 


"""

In [ ]:
reference_structure = {
    "type": "object",
    "properties": {
        "references": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "title": {"type": "string"},
                    "authors": {"type": "string"},
                    "year": {"type": "string"},
            }
        },
    }
}
}

In [59]:
prompt = """convert these references into json format: 'Hu, L. T., & Bentler, P. M. (1999). Cut-off criteria for fit indexes in covariance structure analysis: Conventional criteria versus new alternatives. Structural Equation Modeling: A Multidisciplinary Journal, 6 (1), 1-55. https://doi.org/10.1080/10705519909540118'"""

In [ ]:
builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=reference_structure,
    prompt=format_ref,
    max_string_token_length=300,
    )

In [87]:
out = builder()

In [88]:
out

{'references': [{'id': '1',
   'title': 'The Great Gatsby',
   'authors': 'F. Scott Fitzgerald',
   'year': '1925'}]}

In [34]:
data = json.loads(answer)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [98]:
builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=test,
    prompt=answer)

In [99]:
out = builder()

In [100]:
out

{'references': {'ref_id': 'ref1',
  'title': 'Title of the first paper',
  'author': 'Allen, Rush',
  'year': '2001'}}

In [89]:
highlight_values(output)

{
  ref: {
    title: "Allen, T. D., & Rush",
    author: "Allen, T. D., & Rush",
    year: "2001"
  }
}


In [96]:
parsed_data = []
for line in answer.split('\n'):
    if line.startswith('ref1'):
        parts = line.split(',')
        ref_id = parts[0].split(':')[0].strip()
        title = parts[0].split(':')[1].strip()
        authors = parts[1].split(':')[1].replace('[', '').replace(']', '').strip().split(' ')
        year = int(parts[2].split(':')[1].strip())
        
        parsed_data.append({
            "ref_id": ref_id,
            "title": title,
            "authors": authors,
            "year": year
        })

# Convert to JSON
json_output = json.dumps({"references": parsed_data}, indent=4)

print(json_output)

{
    "references": []
}


In [89]:
answer['references']

TypeError: string indices must be integers

In [92]:
for i in answer:
    print('test')

ValueError: not enough values to unpack (expected 2, got 1)

In [75]:
answer.replace("  ", "")

' ```json\n{\n"message": "Here are the references :)",\n"references": [\n{\n"ref_id": "ref1",\n"title": "Title of the first paper",\n"authors": ["Allen", "Rush"],\n"year": 2001\n},\n{\n"ref_id": "ref2",\n"title": "Title of the second paper",\n"authors": ["Aziz", "Pittman", "Wuensch"],\n"year": 2020\n},\n{\n"ref_id": "ref3",\n"title": "Title of the third paper",\n"authors": ["Balducci", "Cecchin", "Fraccaroli", "Schaufeli"],\n"year": 2012\n}\n]\n}\n```'

In [78]:
import json
from pathlib import Path
from pprint import pprint

In [91]:
data = json.loads(answer)

# Now you can iterate over the 'references' array
for ref in data['references']:
    print('test')

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [83]:
data

' ```json\n{\n  "message": "Here are the references :)",\n  "references": [\n    {\n      "ref_id": "ref1",\n      "title": "Title of the first paper",\n      "authors": ["Allen", "Rush"],\n      "year": 2001\n    },\n    {\n      "ref_id": "ref2",\n      "title": "Title of the second paper",\n      "authors": ["Aziz", "Pittman", "Wuensch"],\n      "year": 2020\n    },\n    {\n      "ref_id": "ref3",\n      "title": "Title of the third paper",\n      "authors": ["Balducci", "Cecchin", "Fraccaroli", "Schaufeli"],\n      "year": 2012\n    }\n  ]\n}\n```'

In [58]:
answer

' ```json\n{\n  "message": "Here are the references :)",\n  "references": [\n    {\n      "ref_id": "ref1",\n      "title": "Title of the first paper",\n      "authors": ["Allen", "Rush"],\n      "year": 2001\n    },\n    {\n      "ref_id": "ref2",\n      "title": "Title of the second paper",\n      "authors": ["Aziz", "Pittman", "Wuensch"],\n      "year": 2020\n    },\n    {\n      "ref_id": "ref3",\n      "title": "Title of the third paper",\n      "authors": ["Balducci", "Cecchin", "Fraccaroli", "Schaufeli"],\n      "year": 2012\n    }\n  ]\n}\n```'

In [59]:
data = json.loads(answer)

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

## Parsing only the reference page:

In [20]:
import PyPDF2

def extract_references(pdf_path):
    pdf_file = open(pdf_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    references = ""
    capture = False

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        
        # Check if text extraction is possible
        if not text:
            print(f"No text found on page {page_num}")
            continue

        if 'References' in text or 'Bibliography' in text or 'references' in text:
            # Here you can split the text and start capturing the references section
            # This assumes that 'References' or 'Bibliography' is a unique heading
            # You may need to refine this logic depending on the actual PDF layout
            parts = text.split('References', 1)
            if len(parts) > 1:
                capture = True
                references += parts[1]
            else:
                parts = text.split('Bibliography', 1)
                if len(parts) > 1:
                    capture = True
                    references += parts[1]
        elif capture:
            # Keep capturing until we decide we're done (which is tricky to determine automatically)
            references += text

    pdf_file.close()
    return references

# Path to your PDF
pdf_path = "1576_5962_jwop_39_2_0055.pdf"
references_section = extract_references(pdf_path)

# Save or print the extracted references section
print(references_section)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

Allen, T. D., & Rush, M. C. (2001). The influence of ratee gender 
on ratings of organizational citizenship behavior. Journal of 63
HWI, Servant Leadership, OCB and CWB in Italy
Applied Social Psychology, 31 (12), 2561-2587.  https://doi.
org/10.1111/j.1559-1816.2001.tb00191.x  
Andreassen, C. S., Nielsen, M. B., Pallesen, S., & Gjerstad, J. (2019). The 
relationship between psychosocial work variables and workaholism: 
Findings from a nationally representative survey. International Journal 
of Stress Management, 26 (1), 1-10. https://doi.org/10.1037/str0000073   
Aziz, S., Pittman, C., & Wuensch, K. (2020). Workaholism and organizational 
citizenship behaviors: Expl

In [24]:
prompt = """
You need to extract the reference from the text and re-structure it in this format: 
{ref1:{"name":"X", "author":"Y",....... }}- ...

Re organize the following from:
"""

## Using Anystyle and Arxiv Search API

In [ ]:
# imports:
import bibtexparser

import os
import PyPDF2
import arxiv
import time
from urllib.error import HTTPError

import arxiv
from tqdm import tqdm

#### Verify that the query and the result match to avoid unwanted data

In [ ]:
def is_close_match(result_title, query_title, result_author, query_author):
    return (query_title.lower() in result_title.lower()) and (query_author.lower() in result_author.lower())

#### *Brute force* Arxiv search

In [ ]:
def arxiv_search(title,authors):    
    for titles in title:
        for author in authors:
        
            search_results = arxiv.Search(
                query= f"au:{author} AND ti:{titles}",
                max_results=1,
            )


            for result in tqdm(search_results.results()):
                    result_title = result.title
                    result_author = ', '.join([a.name for a in result.authors])
                    print('title', result_title, 'authors', result_author)

                    if is_close_match(result_title, titles, result_author, author):
                        try:
                            result.download_pdf(dirpath="./pdfs")
                            break
                        except FileNotFoundError:
                            print("file not found")
                            break
                        except HTTPError:
                            print("forbidden")
                            break
                        except ConnectionResetError as e:
                            print("connection reset by peer")

                            # wait for some time before retrying the connection
                            time.sleep(5)

#### Processing of references after anystyle parsing to better search capabilities

In [ ]:
with open('../arxiv_reference_parsing/json/2304.01597v1.Unsupervised_Improvement_of_Factual_Knowledge_in_Language_Models.bib') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

def clean_author_names(author_string):
    cleaned_authors = []
    for author in author_string.split(" and "):
        parts = [part for part in author.replace(',', '').split() if len(part.replace('.', '')) > 1 and not all(c.isupper() for c in part.replace('.', ''))]
        cleaned_authors.extend(parts)

    return cleaned_authors[:4]

def clean_title(title_string):
    cleaned_title = [part.replace('title:', '').strip() for part in title_string.split(":")]
    return cleaned_title

references = []
for entry in bib_database.entries:
    authors = clean_author_names(entry.get("author", ""))
    title = clean_title(entry.get("title", "No title"))
    references.append({"authors": authors, "title": title})

#### Search on Arxiv for each references

In [ ]:
for ref in references:
    print(f"Searching for Title: {ref['title']}, Author: {ref['authors']}")
    arxiv_search(ref['title'], ref['authors'])

### Main limits of Arxiv search

* Reference parsing
* Reference formatting and cleaning
* Does not relly much on state-of-art AI
* Sub-optimal search system (almost brute force)
* Limited search capability (only Arxiv API) 

### Improvements for the Arxiv search


* Reference parsing:
    * Automate anystyle paring
    * Improve the anystyle library
    * Use fine-tuned CV model (YOLOv8) for detection and extraction of references
    * Grobid library
* Reference cleaning:
    * Explore the limits of arxiv search API (ex: doesn't support :'s)
    * Test with additional information such as Journal, DOI
    * Improve the structure of the data that is passed to the search engine
* Arxiv paper parsing:
    * Add error handling
    * Move on to next reference when a paper is found to reduce the search space
    * Improve the nested loops
    * Recursive parsing
    * Add necessary conditions to optimize the search process
* Paralellization:
    * Reference cleaning and paper parsing can be optimized using Ray actors
* Weaviate:
    * Parse the text from the fetched papers concurrently
    * Explore best practices to store scientific paper in a VDB (ex: paper content, metadata ...)

In [ ]:
from PyPDF2.errors import PdfStreamError
import pypdf
def parse_pdf():    
    documents = []
    for file in os.listdir('./arxiv_pdfs/'):
        if file.endswith('.pdf'):
            pdf_path = os.path.join('./arxiv_pdfs', file)
            try:
                loader = PyPDFLoader(pdf_path)
                documents.extend(loader.load())
            except pypdf.errors.PdfStreamError as e:
                print(f"Skipping file {file} due to error: {e}")
                continue  # Skip this file and continue with the next one
        elif file.endswith('.txt'):
            text_path = os.path.join('./arxiv_pdfs', file)
            try:
                loader = TextLoader(text_path)
                documents.extend(loader.load())
            except Exception as e:
                print(f"Error in file {file}: {e}")
                continue
    return documents

## Arxiv pipeline

#### Installs

In [ ]:
# Install Ruby and Gem

# sudo gem install anystyle-cli

# anystyle -f json find ../weaviate_tests/arxiv_pdfs/2304.01597v1.Unsupervised_Improvement_of_Factual_Knowledge_in_Language_Models.pdf json

In [2]:
# imports

import subprocess
import json
import bibtexparser

import os
import PyPDF2
import arxiv
import time
from urllib.error import HTTPError

import arxiv
from tqdm import tqdm

from PyPDF2.errors import PdfStreamError
import pypdf
from langchain.document_loaders import PyPDFLoader
from PyPDF2.errors import PdfReadError, PdfStreamError

import weaviate
import json
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from torch import cuda, bfloat16
from langchain.llms import HuggingFacePipeline

from langchain.chains import ChatVectorDBChain,RetrievalQA
import ray

In [3]:
ray.init(num_gpus=1)

/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1403: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-11-22_14-54-33_139407_2648046/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1403: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-11-22_14-54-33_139407_2648046/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1408: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-11-22_14-54-33_139407_2648046/logs/monitor.out' mode='a' encoding='utf-8'>
  self.start_monitor()
/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1408: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-11

Python version:,3.8.10
Ray version:,2.7.1
Dashboard:,http://127.0.0.1:8266


#### Anystyle function

In [49]:
def run_anystyle(input_pdf):
    try:
        command = ['anystyle', '-f', 'bib', 'find', input_pdf, 'bib_files']
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Check for successful execution
        if result.returncode == 0:
            output_file_name = os.path.basename(input_pdf).replace('.pdf', '.' + 'bib')
            output_file_path = os.path.join('./bib_files', output_file_name)
            print('check the path: ', output_file_path)
            return output_file_path
        else:
            return f"Command failed with return code {result.returncode}."

    except subprocess.CalledProcessError as e:
        # Handle errors
        return f"An error occurred: {e.stderr}"

#### Process the reference from the bib file

In [5]:
def process_bib_files(file):
    try:
        with open(file) as bibtex_file:
            bib_database = bibtexparser.load(bibtex_file)

        references = []
        for entry in bib_database.entries:
            authors = clean_author_names(entry.get("author", ""))
            title = clean_title(entry.get("title", "No title"))
            references.append({"authors": authors, "title": title})

        return references
    
    except FileNotFoundError:
        return "BibTeX file not found."
    except Exception as e:
        return f"An error occurred: {e}"

In [6]:
def clean_author_names(author_string):
    cleaned_authors = []
    for author in author_string.split(" and "):
        parts = [part for part in author.replace(',', '').split() if len(part.replace('.', '')) > 1 and not all(c.isupper() for c in part.replace('.', ''))]
        cleaned_authors.extend(parts)

    return cleaned_authors[:4]

def clean_title(title_string):
    cleaned_title = [part.replace('title:', '').strip() for part in title_string.split(":")]
    return cleaned_title

In [7]:
def is_close_match(result_title, query_title, result_author, query_author):
    return (query_title.lower() in result_title.lower()) and (query_author.lower() in result_author.lower())

#### Arxiv Search

In [27]:
def arxiv_search(titles, authors, dir):
    for title in titles:
        for author in authors:
            try:
                search_query = f"au:{author} AND ti:{title}"
                search_results = arxiv.Search(query=search_query, max_results=1)

                for result in tqdm(search_results.results()):
                    result_title = result.title
                    result_author = ', '.join([a.name for a in result.authors])
                    print(f"Title: {result_title}, Authors: {result_author}")

                    if is_close_match(result_title, title, result_author, author):
                        try:
                            result.download_pdf(dirpath=dir)
                            return  # Exit the loop once a match is found and downloaded
                        except FileNotFoundError:
                            print("File not found.")
                        except HTTPError:
                            print("Access forbidden.")
                        except ConnectionResetError:
                            print("Connection reset by peer. Retrying in 5 seconds.")
                            time.sleep(5)
                            continue  # Retry the current iteration
                break  # Break the inner loop if a search is completed

            except Exception as e:
                print(f"An error occurred: {e}")
                break  # Break the inner loop on encountering an exception


#### Weaviate functions

In [9]:
from weaviate.util import generate_uuid5
import time

@ray.remote(num_gpus=0.1)
class WeaviateRayEmbedder:
    def __init__(self):
        self.time_taken = 0
        self.text_list = []
        self.weaviate_client = weaviate.Client(
            url="http://localhost:8080",   
        )

    def adding_weaviate_document(self, text_lst, collection_name):
        start_time = time.time()
        self.weaviate_client.batch.configure(batch_size=100)

        with self.weaviate_client.batch as batch:
            for text in text_lst:
                    batch.add_data_object(
                        text,
                        class_name=collection_name, 
                        uuid=generate_uuid5(text),
        )
        self.text_list.append(text)
        self.time_taken = time.time() - start_time
        return self.text_list

    def get(self):
        return self.lst_embeddings
    
    def get_time_taken(self):
        return self.time_taken

In [10]:
from weaviate.util import generate_uuid5
import time

class WeaviateEmbedder:
    def __init__(self):
        self.time_taken = 0
        self.text_list = []
        self.weaviate_client = weaviate.Client(
            url="http://localhost:8080",   
        )

    def adding_weaviate_document(self, text_lst, collection_name):
        start_time = time.time()
        self.weaviate_client.batch.configure(batch_size=100)

        with self.weaviate_client.batch as batch:
            for text in text_lst:
                    batch.add_data_object(
                        text,
                        class_name=collection_name, 
                        uuid=generate_uuid5(text),
        )
        self.text_list.append(text)
        self.time_taken = time.time() - start_time
        return self.text_list

    def get(self):
        return self.lst_embeddings
    
    def get_time_taken(self):
        return self.time_taken

In [11]:
def weaviate_serialize_document(doc, title):
        return {
            "page_content": doc.page_content,
            "document_title": title,
        }

In [12]:
actors = [WeaviateEmbedder.remote() for _ in range(num_actors)]

results = [actor.adding_weaviate_document.remote(doc_part, "WokToWalk") for actor, doc_part in zip(actors, doc_parts)]

NameError: name 'num_actors' is not defined

In [13]:
def parse_pdf():    
    documents = []
    for file in os.listdir('./arxiv_pdfs/'):
        if file.endswith('.pdf'):
            pdf_path = os.path.join('./arxiv_pdfs', file)
            try:
                loader = PyPDFLoader(pdf_path)
                documents.extend(loader.load())
            except pypdf.errors.PdfStreamError as e:
                print(f"Skipping file {file} due to error: {e}")
                continue  # Skip this file and continue with the next one
        elif file.endswith('.txt'):
            text_path = os.path.join('./arxiv_pdfs', file)
            try:
                loader = TextLoader(text_path)
                documents.extend(loader.load())
            except Exception as e:
                print(f"Error in file {file}: {e}")
                continue
    return documents

In [14]:
def weaviate_serialize_document(doc):
        document_title = doc.metadata.get('source', '').split('/')[-1]
        return {
            "page_content": doc.page_content,
            "document_title": document_title,

        }

In [15]:
def weaviate_split_multiple_pdf(docs):    
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    text_docs = text_splitter.split_documents(docs)

    serialized_docs = [
                weaviate_serialize_document(doc) 
                for doc in text_docs
                ]
    return serialized_docs	

In [16]:
def split_document(docs, doc_name):        

        loader = PyPDFLoader(docs)

        documents = loader.load()

        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

        text_docs = text_splitter.split_documents(documents)
        serialized_docs = [
            weaviate_serialize_document(doc,doc_name) 
            for doc in text_docs
            ]
        return serialized_docs	

In [17]:
def process_and_remove_pdfs(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)

            # Apply the split_document function
            doc_name = filename[:-4]  # Remove '.pdf' from filename to get the document name
            try:
                serialized_docs = split_document(file_path, doc_name)
                # Process serialized_docs as needed
                print(f"Processed {filename}")

                # Remove the PDF file after processing
                os.remove(file_path)
                print(f"Removed {filename}")
                
            except Exception as e:
                print(f"Error processing {filename}: {e}")


In [ ]:
dum = "pdfs"

os.listdir(dum)

['1303.5778v1.Speech_Recognition_with_Deep_Recurrent_Neural_Networks.pdf',
 '1212.5701v1.ADADELTA__An_Adaptive_Learning_Rate_Method.pdf',
 '1301.3584v7.Revisiting_Natural_Gradient_for_Deep_Networks.pdf',
 '1206.1106v2.No_More_Pesky_Learning_Rates.pdf',
 '1207.0580v1.Improving_neural_networks_by_preventing_co_adaptation_of_feature_detectors.pdf',
 '1308.0850v5.Generating_Sequences_With_Recurrent_Neural_Networks.pdf']

In [60]:
def parse_pdf(dir):    
    documents = []
    for file in os.listdir(dir):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(dir, file)
            try:
                loader = PyPDFLoader(pdf_path)
                documents.extend(loader.load())
            except pypdf.errors.PdfStreamError as e:
                print(f"Skipping file {file} due to error: {e}")
                continue  # Skip this file and continue with the next one
        elif file.endswith('.txt'):
            text_path = os.path.join(dir, file)
            try:
                loader = TextLoader(text_path)
                documents.extend(loader.load())
            except Exception as e:
                print(f"Error in file {file}: {e}")
                continue
    return documents

In [19]:
def divide_workload(num_actors, documents):
    docs_per_actor = len(documents) // num_actors

    doc_parts = [documents[i * docs_per_actor: (i + 1) * docs_per_actor] for i in range(num_actors)]

    if len(documents) % num_actors:
        doc_parts[-1].extend(documents[num_actors * docs_per_actor:])

    return doc_parts

In [ ]:
input_pdf_path = "../weaviate_tests/arxiv_pdfs/2304.01597v1.Unsupervised_Improvement_of_Factual_Knowledge_in_Language_Models.pdf"
output_directory = "bib"  # Specify the desired output directory
anystyle_output = run_anystyle(input_pdf_path)
print(anystyle_output)

./bib_files/2304.01597v1.Unsupervised_Improvement_of_Factual_Knowledge_in_Language_Models.bib


In [20]:
def weaviate_embedding(text, cls):
    embedder = WeaviateEmbedder()
    embedder.adding_weaviate_document(text, cls)


In [21]:
def weaviate_ray_embedding(text,cls):
    actor_workload = divide_workload(4, text)
    actors = [WeaviateRayEmbedder.remote() for _ in range(4)]
    [actor.adding_weaviate_document.remote(doc_part, cls) for actor, doc_part in zip(actors, actor_workload)]

In [59]:
import shutil
import os
import re

def merge_all_pdfs_into_final_dir(final_dir):
    if not os.path.exists(final_dir):
        os.makedirs(final_dir)

    # Regular expression to match iteration directories
    iter_dir_pattern = re.compile(r'^iteration_\d+$')

    # List all directories that match the iteration pattern
    all_iter_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and iter_dir_pattern.match(d)]

    for iter_dir in all_iter_dirs:
        for pdf_file in os.listdir(iter_dir):
            if pdf_file.endswith('.pdf'):
                src_file_path = os.path.join(iter_dir, pdf_file)
                dest_file_path = os.path.join(final_dir, pdf_file)

                # Check for filename conflicts and rename if necessary
                file_index = 1
                base_name, extension = os.path.splitext(dest_file_path)
                while os.path.exists(dest_file_path):
                    dest_file_path = f"{base_name}_{file_index}{extension}"
                    file_index += 1

                shutil.move(src_file_path, dest_file_path)

In [63]:
import shutil


def arxiv_pipeline(input_pdf, cls, ray=False, recursive=False, iteration = None):
    
        """Process all on one actor"""

        current_iter = 1

        print('check before recursive')

        if not recursive:
            anystyle_output = run_anystyle(input_pdf)
            parsed_data = process_bib_files(anystyle_output)
            for ref in parsed_data:
                arxiv_search(ref['title'], ref['authors'])


            print('check not recursive')
            parsed_text = parse_pdf()
            serialized_text = weaviate_split_multiple_pdf(parsed_text)

            if ray == False:
                print('success split with no ray')
            # calling the weaviate embedder
                weaviate_embedding(serialized_text, cls)

            elif ray is True:
                print('splt with ray')
                weaviate_ray_embedding(serialized_text, cls)


            for filename in os.listdir('./pdfs/'):
                file_path = os.path.join('./pdfs', filename)
                if os.path.isfile(file_path) and filename.endswith(".pdf"):
                    os.remove(file_path)
                    print(f"Removed {filename}")

        if recursive and iteration > 0:
            while current_iter <= iteration:
                print('test while loop 1')
                if current_iter == 1:
                    print('test while loop 2')
                    iter_dir = f'./iteration_{current_iter}'
                    if not os.path.exists(iter_dir):
                        os.makedirs(iter_dir)
                    print('test while loop 3')
                    anystyle_output = run_anystyle(input_pdf)
                    parsed_data = process_bib_files(anystyle_output)
                    for ref in parsed_data:
                        arxiv_search(ref['title'], ref['authors'], iter_dir)
                    print('test while loop 4')
                    current_iter += 1
                    
                elif current_iter >= 2:
                    print('test while loop 5')
                    iter_dir = f'./iteration_{current_iter}'
                    if not os.path.exists(iter_dir):
                        os.makedirs(iter_dir)
                   
                    previous_dir = f'./iteration_{current_iter - 1}'
                    print('checking the directories prev and current and current iteration:', iter_dir, previous_dir, current_iter)
                    pdf_files = [f for f in os.listdir(previous_dir) if f.endswith('.pdf')]
                    print('pdf files in iterdir', pdf_files)
                    for pdf_file in pdf_files:
                        full_path = os.path.join(previous_dir, pdf_file)
                        print('pdf file:', full_path)
                        anystyle_output = run_anystyle(full_path)
                        print('check anystyle bib', anystyle_output)
                        
                        parsed_data = process_bib_files(anystyle_output)
                        for ref in parsed_data:
                            if isinstance(ref, dict) and 'title' in ref and 'authors' in ref:
                                arxiv_search(ref['title'], ref['authors'], iter_dir)
                            else:
                                print(f"Unexpected format of reference: {ref}")
                    current_iter += 1
            
            final_directory = './final_pdfs'
            merge_all_pdfs_into_final_dir(final_directory)
            parsed_text = parse_pdf(final_directory)
            serialized_text = weaviate_split_multiple_pdf(parsed_text)
            if ray == False:
                print('success split with no ray')
            # calling the weaviate embedder
                weaviate_embedding(serialized_text, cls)

            elif ray is True:
                print('splt with ray')
                weaviate_ray_embedding(serialized_text, cls)


In [64]:
arxiv_pipeline("./1412.6980v9.Adam__A_Method_for_Stochastic_Optimization.pdf", "new_class_ray", ray=True, recursive=True, iteration=3)

check before recursive
test while loop 1
test while loop 2
test while loop 3


/tmp/ipykernel_2648046/992132040.py:8: DeprecationWarning: The '(Search).results' method is deprecated, use 'Client.results' instead
  for result in tqdm(search_results.results()):


check the path:  ./bib_files/1412.6980v9.Adam__A_Method_for_Stochastic_Optimization.bib


0it [00:00, ?it/s]
1it [00:00,  1.82it/s]


Title: Fisher Information and Natural Gradient Learning of Random Deep Networks, Authors: Shun-ichi Amari, Ryo Karakida, Masafumi Oizumi


...

0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.76it/s]


Title: A Quasi-Newton Approach to Nonsmooth Convex Optimization Problems in Machine Learning, Authors: Jin Yu, S. V. N. Vishwanathan, Simon Guenter, Nicol N. Schraudolph


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.28s/it]

Title: A General Algorithm for Solving Rank-one Matrix Sensing, Authors: Lianke Qin, Zhao Song, Ruizhe Zhang


splt with ray


In [65]:
weaviate_client = weaviate.Client(url="http://localhost:8080")

In [66]:
weaviate_client.data_object.get(class_name='new_class_ray')

{'deprecations': [],
 'objects': [{'class': 'New_class_ray',
   'creationTimeUnix': 1700671887430,
   'id': '0053d892-29fa-5c3a-ae92-b0cd51967ad6',
   'lastUpdateTimeUnix': 1700671887430,
   'properties': {'document_title': '1206.5538v3.Representation_Learning__A_Review_and_New_Perspectives.pdf',
    'page_content': '29\nmodel for visual area V2. In NIPS’07 .\nLee, H., Grosse, R., Ranganath, R., and Ng, A. Y . (2009a). Convolu-\ntional deep belief networks for scalable unsupervised learning of\nhierarchical representations. In ICML’2009 .\nLee, H., Pham, P., Largman, Y ., and Ng, A. (2009b). Unsupervised\nfeature learning for audio classiﬁcation using convolutional deep\nbelief networks. In NIPS’2009 .\nLin, Y ., Tong, Z., Zhu, S., and Yu, K. (2010). Deep coding network.\nInNIPS’2010 .\nLowe, D. (1999). Object recognition from local scale invariant\nfeatures. In ICCV’99 .\nMallat, S. (2012). Group invariant scattering. Communications on\nPure and Applied Mathematics .\nMarlin, B. and d

In [82]:
weaviate_client.query.aggregate("New_class_ray").with_meta_count().do()

{'data': {'Aggregate': {'New_class_ray': [{'meta': {'count': 976}}]}}}

In [68]:
ish = weaviate_client.query.get(class_name='new_class_ray', properties='document_title')

In [73]:
ish.do()

{'data': {'Get': {'New_class_ray': [{'document_title': '1206.5538v3.Representation_Learning__A_Review_and_New_Perspectives.pdf'},
    {'document_title': '1206.1106v2.No_More_Pesky_Learning_Rates_2.pdf'},
    {'document_title': '1107.2490v2.Towards_Optimal_One_Pass_Large_Scale_Learning_with_Averaged_Stochastic_Gradient_Descent.pdf'},
    {'document_title': '1301.3584v7.Revisiting_Natural_Gradient_for_Deep_Networks.pdf'},
    {'document_title': '1308.0850v5.Generating_Sequences_With_Recurrent_Neural_Networks.pdf'},
    {'document_title': '1412.2306v2.Deep_Visual_Semantic_Alignments_for_Generating_Image_Descriptions.pdf'},
    {'document_title': '1108.3298v1.A_Machine_Learning_Perspective_on_Predictive_Coding_with_PAQ.pdf'},
    {'document_title': '1801.07736v3.MaskGAN__Better_Text_Generation_via_Filling_in_the______.pdf'},
    {'document_title': '1308.0850v5.Generating_Sequences_With_Recurrent_Neural_Networks_1.pdf'},
    {'document_title': '1111.4259v1.Krylov_Subspace_Descent_for_Deep_L

In [78]:
test = weaviate_client.query.aggregate(class_name='New_class_ray')

In [79]:
test.do()

{'errors': [{'locations': [{'column': 25, 'line': 1}],
   'message': 'Syntax Error GraphQL request (1:25) Unexpected empty IN {}\n\n1: {Aggregate{New_class_ray{}}}\n                           ^\n',
   'path': None}]}

In [ ]:


"""

            for i in range(iteration):
                for filename in os.listdir('./pdfs/'):
                        file_path = os.path.join('./pdfs', filename)
                        if os.path.isfile(file_path) and filename.endswith(".pdf"):
                            run_anystyle(file_path)
                            parsed_data = process_bib_files(anystyle_output)
                            for ref in parsed_data:
                                arxiv_search(ref['title'], ref['authors'])
                            print('success search')

                            parsed_text = parse_pdf()
                            serialized_text = weaviate_split_multiple_pdf(parsed_text)
                            os.remove(file_path)

                if ray == False:
                        print('success split with no ray')
                    # calling the weaviate embedder
                        embedder = WeaviateEmbedder()
                        embedder.adding_weaviate_document(serialized_text, cls)

                elif ray is True:
                        print('splt with ray')
                        actor_workload = divide_workload(4, serialized_text)
                        actors = [WeaviateRayEmbedder.remote() for _ in range(4)]
                        [actor.adding_weaviate_document.remote(doc_part, cls) for actor, doc_part in zip(actors, actor_workload)]
                        if recursive:
                            print('recursive with ray')
                            return arxiv_pipeline(input_pdf, cls, ray=True)

                    for filename in os.listdir('./pdfs/'):
                        file_path = os.path.join('./pdfs', filename)
                        if os.path.isfile(file_path) and filename.endswith(".pdf"):
                            run_anystyle(file_path)
                            os.remove(file_path)
                            print(f"Removed {filename}")
    
    """

In [44]:
weaviate_client.data_object.get(class_name='test_no_rec')

{'deprecations': [],
 'objects': [{'class': 'Test_no_rec',
   'creationTimeUnix': 1700659853728,
   'id': '0124446f-5348-5c80-895d-ea31f24ee1e7',
   'lastUpdateTimeUnix': 1700659853728,
   'properties': {'document_title': '1301.3584v7.Revisiting_Natural_Gradient_for_Deep_Networks.pdf',
    'page_content': 'Erhan, D., Courville, A., Bengio, Y ., and Vincent, P. (2010). Why does unsupervised pre-training help deep\nlearning? In JMLR W&CP: Proc. AISTATS’2010 , volume 9, pages 201–208.\nGonzalez, A. and Dorronsoro, J. (2006). Natural conjugate gradient training of multilayer perceptrons. Artiﬁcial\nNeural Networks ICANN 2006 , pages 169–177.\nHeskes, T. (2000). On natural learning and pruning in multilayered perceptrons. Neural Computation ,12,\n1037–1057.\nHonkela, A., Tornio, M., Raiko, T., and Karhunen, J. (2008). Natural conjugate gradient in variational inference.\nInNeural Information Processing , pages 305–314.\nHonkela, A., Raiko, T., Kuusela, M., Tornio, M., and Karhunen, J. (2010

## Storing data in Weaviate

In [ ]:
def divide_workload(num_actors, documents):
    docs_per_actor = len(documents) // num_actors

    doc_parts = [documents[i * docs_per_actor: (i + 1) * docs_per_actor] for i in range(num_actors)]

    if len(documents) % num_actors:
        doc_parts[-1].extend(documents[num_actors * docs_per_actor:])

    return doc_parts

In [16]:
arxiv_pipeline('./1412.6980v9.Adam__A_Method_for_Stochastic_Optimization.pdf')

/tmp/ipykernel_2599421/1579659243.py:8: DeprecationWarning: The '(Search).results' method is deprecated, use 'Client.results' instead
  for result in tqdm(search_results.results()):
0it [00:00, ?it/s]
1it [00:00,  1.79it/s]


Title: Fisher Information and Natural Gradient Learning of Random Deep Networks, Authors: Shun-ichi Amari, Ryo Karakida, Masafumi Oizumi


1it [00:00,  1.34it/s]


Title: Recent Advances in Convolutional Neural Networks, Authors: Jiuxiang Gu, Zhenhua Wang, Jason Kuen, Lianyang Ma, Amir Shahroudy, Bing Shuai, Ting Liu, Xingxing Wang, Li Wang, Gang Wang, Jianfei Cai, Tsuhan Chen


1it [00:00,  1.73it/s]


...

0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.59s/it]


Title: Towards Understanding Fast Adversarial Training, Authors: Bai Li, Shiqi Wang, Suman Jana, Lawrence Carin


0it [00:00, ?it/s]

Title: ADADELTA: An Adaptive Learning Rate Method, Authors: Matthew D. Zeiler


0it [00:02, ?it/s]
0it [00:00, ?it/s]
